## Demo

In [ ]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split, StaticGraphTemporalSignal, StaticGraphTemporalSignalBatch

In [2]:
loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

In [3]:
a = dataset.__getitem__(1)

In [4]:
a

Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])

In [5]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN
from torch_geometric.nn.pool import global_mean_pool
import numpy as np

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)
        self.batch = torch.as_tensor(np.zeros(16), dtype=torch.int64)


    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        h = global_mean_pool(h, size=1, batch=self.batch)
        return h

In [6]:
torch.as_tensor(np.zeros(20), dtype=torch.int64)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
model = RecurrentGCN(node_features=20)
x = torch.rand(20,4)
x.shape

torch.Size([20, 4])

In [8]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x36 and 52x32)

## mimic data

In [1]:
import argparse
import gc
import logging
import time
from pathlib import Path
from typing import Any, Callable, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader

from src.constants import CONTINUOUS_COVARIATES_PROCESSED, STATIC_COLS, TARGET_COL
from src.dataloader import get_dataloaders
from src.models.temporal_gnn import RecurrentGCN
# from src.models.transformer import TimeSeriesTransformer
from src.utils import (
    EarlyStopping,
    generate_square_subsequent_mask,
    get_patient_indices,
    get_timestamp,
)

GLOBAL_SEED = 123
device = torch.device("cpu")

/home/gaurang/anaconda3/envs/gnn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
fpath_train = "/home/gaurang/glycemic_control/data/glycaemia_project_csvs/processed_data/train_test_splits/v4_patient_split_mini/train_mini.csv"
fpath_valid = "/home/gaurang/glycemic_control/data/glycaemia_project_csvs/processed_data/train_test_splits/v4_patient_split_mini/valid_mini.csv"
fpath_test = "/home/gaurang/glycemic_control/data/glycaemia_project_csvs/processed_data/train_test_splits/v4_patient_split_mini/test_mini.csv"

In [4]:
df_train = pd.read_csv(fpath_train)
df_valid = pd.read_csv(fpath_valid)
df_test = pd.read_csv(fpath_test)

In [5]:
np.random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)

# Hyperparams
lr = 1e-3
num_epochs = 20
batch_size = 4

# Params
enc_seq_len = 6  # length of input given to encoder
output_sequence_length = 1  # how many future glucose values to predict
step_size = 1  # Step size, i.e. how many time steps does the moving window move at each step
batch_first = True

# Define input variables
exogenous_vars = (
    CONTINUOUS_COVARIATES_PROCESSED + STATIC_COLS
)  # Each element must correspond to a column name
input_variables = TARGET_COL + exogenous_vars

input_size = len(input_variables)
num_predicted_features = 1


print(
    f"Time series params: \nInput sequence lenght: {enc_seq_len} \nOutput sequence lenght:"
    f" {output_sequence_length} \nStep size: {step_size}"
)


Time series params: 
Input sequence lenght: 6 
Output sequence lenght: 1 
Step size: 1


In [6]:
dataloader_train, dataloader_valid, dataloader_test = \
    get_dataloaders(df_train=df_train,
                    df_valid=df_valid,
                    df_test=df_test,
                    input_variables=input_variables,
                    target_variable=TARGET_COL,
                    enc_seq_len=enc_seq_len,
                    output_sequence_length=output_sequence_length,
                    step_size=step_size,
                    batch_size=batch_size)

Number of training samples: 59 
Number of valid samples: 31 
Number of test samples: 59


In [10]:
next(iter(dataloader_test)).ptr

tensor([ 0, 16, 32, 48, 64])

In [8]:
model = RecurrentGCN(node_features = enc_seq_len, batch_size=batch_size)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in range(100):
    cost = total_cost = 0
    for time, snapshot in enumerate(dataloader_train):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, snapshot.batch)
        cost = torch.mean((y_hat - snapshot.y)**2)
        total_cost += cost.item()
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()
    if (epoch + 1) % 25 == 0:
        print(f"Epoch {epoch} : loss {total_cost}")

Epoch 24 : loss 9.450046638958156
Epoch 49 : loss 7.678094385191798
Epoch 74 : loss 6.808877747505903
Epoch 99 : loss 6.472988476976752


In [9]:
model.eval()
cost = 0
for time, snapshot in enumerate(dataloader_valid):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, snapshot.batch)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 10.3301


In [19]:
model.eval()
cost = 0
preds = []
actual = []
for time, snapshot in enumerate(dataloader_test):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr).detach().numpy()
    rescaled_y_hat = scaler_y.inverse_transform(y_hat)
    preds.append(rescaled_y_hat[0][0])
    actual.append(snapshot.y.detach().numpy()[0])

In [20]:
pd.DataFrame(data=zip(preds, actual), columns=['pred', 'actual'])

pred  actual
0   135.970352   131.0
1   119.439888   129.0
2   109.981560   129.0
3   112.259453   114.0
4   119.747116   115.0
5   113.877556   110.0
6   106.873589    92.0
7   106.339058   130.0
8   131.561691   100.0
9   119.392303    97.0
10  121.316261    99.0
11  110.647560   104.0
12  111.856636   119.0
13  117.745346    99.0
14  110.688629   121.0
15   98.470444   103.0
16   89.170151    97.0
17  101.768234   129.0
18  111.262161   121.0
19  107.151085   110.0
20  120.139748   133.0
21   80.881836   160.0
22   90.282860   174.0
23   93.783768   133.0
24  131.783981   129.0
25  127.790581   112.0
26  111.712555   104.0
27  112.988762    88.0
28  113.042053   105.0
29  120.939484   179.0
30  139.574615   167.0
31  135.070145   160.0
32  116.964729   107.0
33  102.183525    80.0
34  111.557671   113.0
35  116.338921   139.0
36  122.264458   103.0
37  108.960144    85.0
38  104.927650    84.0
39  109.152550    62.0
40  110.234749    78.0
41  113.927521    89.0
42  116.419426   103.0

In [63]:
df_res = df_test[TARGET_COL]
df_res['y_hat'] = ys
df_res

ValueError: Length of values (53) does not match length of index (77)